Chatbot com IA generativa com Gemini -----------------------------------------------------------------------

Gera respostas com base apenas no modelo da IA (Gemini, GPT, Claude, etc). Ele usa o conhecimento pré-treinado da IA, nesse caso, eu utilizei a API do Gemini para alimentar o meu chatbot.

In [ ]:
!pip install -q -U google-generativeai

In [ ]:

import google.generativeai as genai

É importante nunca deixar a sua chave API pública, é uma questão de segurança!

In [ ]:
from google.colab import userdata

# Lê a chave do ambiente ou direto do código (troque conforme método escolhido)
# GOOGLE_API_KEY = "SUA_CHAVE_AQUI"  # <-- método simples
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')  # <-- método seguro
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
#Aconselho verificar os modelos para evitar erros
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

In [ ]:
model = genai.GenerativeModel("gemini-2.5-pro")

In [ ]:
#Teste rápido
response = model.generate_content("Quem criou o Gemini?")
print("Resposta:", response.text)

In [ ]:
#Iniciação do chat interativo
chat = model.start_chat(history=[])

In [ ]:
#Laço de repetição para verificação garantir uma boa "interface" e um funcionamento bacana
while True:
    prompt = input("\nVocê: ")
    if prompt.lower() in ["fim", "sair", "exit"]:
        print("Encerrando chat...")
        break

    response = chat.send_message(prompt)
    print("Gemini:", response.text)

Alguns testes -----------------------------------------------------------------------------------------------------

In [ ]:
#chat = model.start_chat(history=[])
#prompt = input("Esperando o prompt: ")
#while prompt != "fim":
#response = chat.send_message(prompt)
#print(response.text)
#prompt = input("do que é feita o planeta terra?")

Chatbot com RAG (Retrieval-Augmented Generation) -----------------------------------------------------------------

Antes de responder, o bot faz uma busca em uma base de dados, documentos, PDFs, sites, etc — e usa esse conteúdo real como contexto para gerar a resposta.

Um repositório de conhecimento
(pode ser uma pasta com PDFs, um banco de dados, ou até um site).

Um módulo de busca semântica ou embedding, ex:

faiss, chromadb, langchain, llama_index
pra fazer busca vetorial (comparar significado de textos).

Um pipeline tipo esse:
pergunta → gerar embedding → buscar trechos relevantes → juntar contexto → enviar pro modelo (Gemini)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from google.generativeai import GenerativeModel

In [ ]:
#Embeddings com Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
#Indexa documentos (exemplo PDF convertido em texto)
vectorstore = FAISS.from_texts(["texto do doc 1", "texto do doc 2"], embeddings)

In [ ]:

#Cria o retriever (busca semântica)
retriever = vectorstore.as_retriever()

In [ ]:
#Cria o modelo generativo
model = GenerativeModel("gemini-2.5-pro")


In [ ]:
#Cria o pipeline RAG
qa = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff"
)

In [ ]:

#Faz uma pergunta contextualizada
resposta = qa.run("O que esse documento diz sobre privacidade de dados?")
print(resposta)